# Model COVID-19 

Paper title: "How many lives can be saved? A global view on the impact of testing, herd immunity and demographics on COVID-19 fatality rates"

Authors: Miguel Sanchez-Romero, Vanessa di Lego, Alexia Prskawetz, and Bernardo L. Queiros

In [4]:
#=
## Necessary packages
using Pkg
Pkg.add("MAT")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("LinearAlgebra")
Pkg.add("SpecialFunctions")
Pkg.add("Interpolations")
Pkg.add("JLD")
Pkg.add("Distributions")
Pkg.add("Random")
=#


## Call packages
using Plots
using MAT
using CSV
using DataFrames
using LinearAlgebra
using SpecialFunctions
using Interpolations
using JLD
using ExcelReaders
using Distributions
using Random

## Dynamic system

To predict the evolution of the number of deaths caused by the epidemic and highlight the importance of testing we extend a standard epidemiological age-structured SEIR (susceptible-exposed-infected-removed) model by (a) accounting for the age-specific mortality rates, (b) explicitly modeling the mortality rate of the COVID-19 epidemics and (c) introducing isolation periods during the incubation period ($E$) and the infectious period ($I$) after testing.  

To account for the differential effect of mortality by age, each state is comprised of 95 ($\Omega$) age-groups. We distinguish vectors and matrices from scalars by using bold letters. The dynamics of our extended age-structured SEIR model are as follows:

\begin{align}
{\dot{\bf S}_t} & = -{\beta}_t{\bf S}_t  - {\bf M}\cdot{\bf S}_t \\
{\dot{\bf E}^u_t}  & = {\beta}_t[1-\pi_E] {\bf S}_t - \sigma {\bf E}^u_t- {\bf M}\cdot{\bf E}^u_t\\
{\dot{\bf E}^d_t}  & =  {\beta}_t\pi_E {\bf S}_t - \sigma {\bf E}^d_t -{\bf M} \cdot{\bf E}^d_t \\
{\dot{\bf I}^u_t}  & =  \sigma [1-\pi_I] {\bf E}^u_t - \nu {\bf I}^u_t-{\bf M}  \cdot{\bf I}^u_t \\
{\dot{\bf I}^d_t}  & =  \sigma  \pi_I {\bf E}^u_t+\sigma{\bf E}^d_t  - \nu {\bf I}^d_t - {\bf M} \cdot {\bf I}^d_t \\
{\dot{\bf R}_t}  & =  \nu [\mathbf{1}-{\bf M}^{c}(\varepsilon)]{\bf I}^u_t +  \nu [\mathbf{1}-{\bf M}^{c}(\varepsilon)]{\bf I}^d_t - {\bf M} \cdot{\bf R}_t \\
{\dot{\bf D}^c_t}  & =  \nu {\bf M}^{c}(\varepsilon)\cdot ({\bf I}^u_t+{\bf I}^d_t)
\end{align}

where ${\beta}_t=\beta [{\sum_{x=0}^{\Omega}{\bf I}^u_t(x)}/{\sum_{x=0}^{\Omega}({\bf S}_t(x)+{\bf E}^u_t(x)+{\bf I}^u_t(x)+{\bf R}_t(x))}]$ is the infection rate and \(\beta\) is the effective transmission rate. ${\bf S}_t$ denotes the vector of susceptible individuals. $\{{\bf E}_t^u,{\bf I}_t^u\}$ denote the vector of exposed and infectious individuals, respectively, who are undetected. $\{{\bf E}^d_t, {\bf I}^d_t\}$ denote the vector of exposed and infectious individuals, respectively, who are detected (tested) and isolated. Note that we denote the state ${\bf I}^d_t$ as the infectious detected people and ${\bf E}^d_t$ as the detected exposed people (those having been in close contact with infected people and therefore been isolated). ${\bf R}_t$ is the vector of recovered individuals and ${\bf D}^{c}_t$ is the vector of deaths from the COVID-19 outbreak. $\mathbf{M}$ and $\mathbf{M}^{c}(\varepsilon)$ are, respectively, diagonal matrices with age-specific death rates not caused by COVID-19 and the estimated age-specific fatality rates for the COVID-19. The term $\varepsilon$ in $\mathbf{M}^{c}(\varepsilon)$ denotes the share of infectious and symptomatic individuals among all the infectious individuals (i.e. also including the asymptomatic cases). We assume that the (true) fatality rate, $\mathbf{M}^{c}(\varepsilon)$, is a fraction $\varepsilon$ of the (observed) case fatality rate. The (true) fatality rate is defined as the ratio between the total number of deaths from COVID-19 and the total number of infectious individuals, whereas the (observed) case fatality rate is the ratio between the total number of deaths from COVID-19 and the total infected individuals who are detected (tested). Thus, the lower the share of symptomatic individuals the lower the (true) fatality rate compared to the (observed) case fatality rate. ${\bf 1}$ denotes the identity matrix. The set of parameters $\{\beta,\pi_E,\pi_I, \nu,\sigma,\varepsilon\}$ denotes the effective transmission rate of the disease, the fraction of people isolated out of the exposed individuals, the fraction of people isolated out of the total infectious individuals, the removal rate, the inverse of the incubation period, and the share of infectious and symptomatic individuals. 


In [2]:
function fdSystem(S,QS,E,Ed,I,Id,R,D,Dcov,Id,M,Mc,β,ν,σ,ϵ,τ1,τ2,tI,t)
    Mcov=ϵ*Mc
    βt=β*(sum(I)/sum(S+E+I+R))

    local πE
    local πI
    local πS
    local ρ # This parameter activating lockdown measures, which has not been considered for the paper
    
    
    # Dynamic system    
    ##### Testing #####
    πS, πE, πI, ρ=fTesting(ϵ,τ1,τ2,tI,t)
    ######################
    
    dS   =-((βt+πS)*Id+M*dt)*S+ρ*QS
    dQS  =πS*Id*S-(ρ*Id+M*dt)*QS
    dE   =βt*(1.0-πE)*Id*S-(σ*Id+M*dt)*E
    dEd  =βt*πE*Id*S-(σ*Id+M*dt)*Ed    
    dI   =σ*(1.0-πI)*Id*E-(ν*Id+M*dt)*I
    dId  =σ*Id*Ed+σ*πI*Id*E-(ν*Id+M*dt)*Id    
    dR   =ν*(Id-Mcov)*I+ν*(Id-Mcov)*Id-dt*M*R
    dD   =M*(S+QS+E+Ed+I+Id+R)*dt
    dDcov=ν*Mcov*I+ν*Mcov*Id

    
    return dS, dQS, dE, dEd, dI, dId, dR, dD, dDcov, βt, tI
    end;

## Isolation and testing


Let $\varepsilon$ be the fraction of infectious people who develop some symptoms and $1-\varepsilon$ those infectious individuals who are asymptomatic. Following  \cite{Chinese20} we assume that among the fraction $\varepsilon$ of infectious individuals, 80\% develop mild symptoms, while 20\% develop critical and severe symptoms. Moreover, we consider that governments do not start isolating individuals until time $t_I$. From time $t_I$ onwards, we consider that governments may commit to find those individuals who are infected. Let the proportion of asymptomatic and positively tested people who become isolated be $\tau_1$. Let the proportion of symptomatic and positively tested people who become isolated be $\tau_2$. Thus, we define the fraction of people who are exposed and isolated as
\begin{gather}
\pi_E\equiv \pi_E(\tau_1,t_I)=
\begin{cases}
0& \text{if $t<t_I$},\\
\tau_1& \text{if $t\geq t_I$},
\end{cases}
\end{gather}
while we define the fraction of people infected who become isolated as
\begin{gather}
\pi_I\equiv \pi_I(\varepsilon,\tau_1,\tau_2,t_I)=
\begin{cases}
0 & \text{if $t<t_I$},\\
\tau_2\varepsilon+\tau_1(1-\varepsilon)& \text{if $t\geq t_I$}.
\end{cases}
\end{gather}
In order to analyze the impact that different degrees of testing and isolation have on controlling the spread of COVID-19 we implement five different testing options, which are summarized in Table~S3. In option (a) only individuals who develop critical or severe symptoms are tested $\{\tau_1=0,\,\tau_2=0.2\}$. In option (b) half of the symptomatic individuals are tested $\{\tau_1=0,\,\tau_2=0.5\}$. In option (c) all the symptomatic individuals are tested $\{\tau_1=0,\,\tau_2=1.0\}$. In option (d) all symptomatic are tested and also one-quarter of asymptomatic individuals $\{\tau_1=0.25,\,\tau_2=1.0\}$. In option (e) all symptomatic are tested and also half of the asymptomatic individuals $\{\tau_1=0.5,\,\tau_2=1.0\}$.

In [1]:
function fTesting(ϵ,τ1,τ2,tI,t)
local πS=0.0
local πE=0.0
local πI=0.0
local ρ =0.0   
        
if  t>=tI
πI=τ2.*ϵ.+τ1*(1.0.-ϵ)
πE=τ1
πS=0.0
ρ =0.0
end
    
return πS, πE, πI, ρ            
end;

## Program

In [3]:
function fDyn(β,ν,σ,ϵ,τ1,τ2,T,P,Id,M,Mc,IC,IA,tI)
S  =zeros(MaxAge,T)
QS =zeros(MaxAge,T)    
E  =zeros(MaxAge,T)
Ed =zeros(MaxAge,T)
I  =zeros(MaxAge,T)
Id =zeros(MaxAge,T)
R  =zeros(MaxAge,T)
D  =zeros(MaxAge,T)
Dc =zeros(MaxAge,T)
dDc=zeros(MaxAge,T)    
βV =zeros(1,T)
TS =zeros(1,T)
Curve=zeros(1,T)    
    
    
Curve[1,1]=1.0    
S[:,1]    =(1.0-HerdI)*P
R[:,1]    =HerdI*P
I[IA[1,1],1]    =I[IA[1,1],1]
TS[1,1]   =0  
tm        =0.0    

    for t=1:(T-1)
    S2, QS2, E2, Ed2, I2, Id2, R2, D2, Dc2, βV[1,t], TS[1,t]=fdSystem(S[:,t],QS[:,t],E[:,t],Ed[:,t],I[:,t],Id[:,t],R[:,t],D[:,t],Dc[:,t],Id,M,Mc,β,ν,σ,ϵ,τ1,τ2,tI,tm);
    S[:,t+1] =S[:,t]+S2
    QS[:,t+1]=QS[:,t]+QS2                
    E[:,t+1] =E[:,t]+E2    
    Ed[:,t+1]=Ed[:,t]+Ed2        
    I[:,t+1] =I[:,t]+I2
    I[IA[t+1,1],t+1] =I[IA[t+1,1],t+1]+IC[t+1,1]
    Id[:,t+1]=Id[:,t]+Id2
    R[:,t+1] =R[:,t]+R2    
    D[:,t+1] =D[:,t]+D2    
    Dc[:,t+1]=Dc[:,t]+Dc2
    dDc[:,t] =Dc2        
    tm+=1.0    
    end
   return S, QS, E, Ed, I, Id, R, D, Dc, βV, TS, dDc 
end;

In [6]:
data = readxl("PopData.xlsx", "Data!A1:E102")
data = DataFrame(data[2:end,:], [:Age, :PxAT, :mxAT, :PxCH, :mxCH]);

***Underlying fatality rates***

\begin{gather}
\log m_x=\beta_0+\beta_1 x+\beta_2 x^2+\epsilon
\end{gather}
which gives the following result in R

Coefficients:

|Coef       |Estimate |Std Error|$t$ value|$Pr(>|t|)$   |
| :---  | ---: | ---: | ---: | ---: |
|       |       |      |      |      |
|(Intercept)   |$-10.51$   | 0.9540   | $-11.013$ |  0.0016 $**$|
|age           |$ 0.1310$  | 0.03361  | $ 3.898$  |  0.0300 $*$ |
|age2          |$-0.003411$| 0.0002780|  $-1.227$ |  0.3073     |


Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1699 on 3 degrees of freedom
Multiple R-squared:  0.994,	Adjusted R-squared:  0.9899 
F-statistic: 246.8 on 2 and 3 DF,  p-value: 0.0004696


In [1]:
# Fitted fatality rates
age=data[1:95,1]
mx=Array{Float64,1}(exp.(-10.5063.+0.1310116.*age.-0.0003411423.*(age.^2.0)));

UndefVarError: UndefVarError: data not defined

In [10]:
# UN data 
CasesPop=CSV.read("un_grad_pop.csv";header=true)
CasesRat=CSV.read("un_grad_rates.csv";header=true)
CasesCou=CSV.read("un_grad_country.csv";header=true);

In [ ]:
# Bayesian Melding parameters
Parameters=CSV.read("BayesianParameters.csv";header=true)
Parameters=DataFrame(Parameters)

In [ ]:
βD =Vector{Float64}(Parameters[:,1])
νD =Vector{Float64}(Parameters[:,2])
σD =Vector{Float64}(Parameters[:,3])
ϵD =Vector{Float64}(Parameters[:,4])

In [11]:
# Factor transformation annual data in daily data
const dt=1/365;

In [1]:
# Testing decisions
τ1D =0.50
τ2D =1.00
# Day of intervention
tID =0
# Herd immunity level
const HerdI=0.25

# Maximum time frame
T  =365

NCases=length(βD)
MaxAge=95

# Imported infectious cases
Random.seed!(123)
ImportedCases=rand(Poisson(10.0),1,T)'
# Age of the imported and infectious cases
ImportedAges =rand(18:65,1,T)'

# Compute the model for the 200 countries
for country=1:200
Px    =Array{Float64,1}(CasesPop[:,1+country])
Mx    =Diagonal(Array{Float64,1}(CasesRat[:,1+country]))
MxCOV =Diagonal(Array{Float64,1}(mx))
Ix    =Matrix{Float64}(I, MaxAge, MaxAge)
Ox    =zeros(Float64, MaxAge, MaxAge);

SMat  =zeros(NCases,T)
QSMat =zeros(NCases,T)
EMat  =zeros(NCases,T)
EdMat =zeros(NCases,T)
IMat  =zeros(NCases,T)
IdMat =zeros(NCases,T)
RMat  =zeros(NCases,T)
DMat  =zeros(NCases,T)
DcMat =zeros(NCases,T)
dDcMat=zeros(NCases,T)    
IfMat =zeros(NCases,T)
NMat  =zeros(NCases,T)
βtMat =zeros(NCases,T)


for k=1:NCases
display(k)    
St, QSt, Et, Edt, It, Idt, Rt, Dt, Dct, βtV, dDct =fDyn(βD[k],νD[k],σD[k],ϵD[k],τ1D,τ2D,T,Px,Ix,Mx,MxCOV,ImportedCases,ImportedAges,tID)
    SMat[k,:] =sum(St,dims=1)
    QSMat[k,:]=sum(QSt,dims=1)    
    EMat[k,:] =sum(Et,dims=1)
    EdMat[k,:]=sum(Edt,dims=1)
    IMat[k,:] =sum(It,dims=1)
    IdMat[k,:]=sum(Idt,dims=1)
    RMat[k,:] =sum(Rt,dims=1)    
    DMat[k,:] =sum(Dt,dims=1)    
    DcMat[k,:]=sum(Dct,dims=1)
   dDcMat[k,:]=sum(dDct,dims=1)        
    βtMat[k,:]=βtV
    TSMat[k,:]=TSV

end

FinDat=cat(dims=3,SMat,QSMat,EMat,EdMat,IMat,IdMat,RMat,DMat,DcMat,dDcMat,IfMat,NMat,βtMat,TSMat)
Case=["S","QS","E","Ed","I","Id","R","D","Dc","dDc","Beta","Ts"]
MatrixRes=Array{Any}(nothing, T*NCases*length(Case),8)
iter=1
for t=1:T
    for k=1:NCases
        for i=1:length(Case)
         MatrixRes[iter,1]=CasesCou[country,1]  
         MatrixRes[iter,2]=τ1D  
         MatrixRes[iter,3]=τ2D
         MatrixRes[iter,4]=tID                  
         MatrixRes[iter,5]=t
         MatrixRes[iter,6]=k
         MatrixRes[iter,7]=Case[i]
         MatrixRes[iter,8]=FinDat[k,t,i]
        iter+=1
        end
    end
end
  
#Uncomment for storing the information in a csv file   
#CSV.write("foo"*CasesCou[country,1]*".csv",DataFrame(MatrixRes),delim=',';append=false);       
end;

UndefVarError: UndefVarError: CSV not defined